# Step 6: V03 — Duration consistency (stellar-density sanity check)

Goal: check whether the observed transit duration is broadly consistent with a planet transiting the target star given the stellar parameters.

Why this matters:
- Large inconsistencies can indicate an incorrect host star assumption, an incorrect ephemeris, or non-planet scenarios.
- Moderate tension can also arise from impact parameter/eccentricity; this check is a **sanity check**, not a validator by itself.


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

ds = sh.load_dataset()
lc = sh.stitch_pdcsap(ds)
depth_ppm, _ = sh.estimate_depth_ppm(lc)
candidate = sh.make_candidate(depth_ppm)

session = sh.make_session(stitched=lc, candidate=candidate, network=False)
r = session.run('V03')

print(
    json.dumps(
        {
            'status': r.status,
            'flags': r.flags,
            'metrics': r.metrics,
        },
        indent=2,
        sort_keys=True,
    )
)


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "flags": [],
  "metrics": {
    "density_corrected": true,
    "duration_hours": 4.046,
    "duration_ratio": 0.632,
    "expected_duration_hours": 6.3971,
    "expected_duration_solar": 4.4083,
    "period_days": 14.2424,
    "stellar_density_solar": 0.3272,
    "stellar_mass": 1.47,
    "stellar_radius": 1.65
  },
  "status": "ok"
}
```

</details>


In [ ]:
# Plot V03: Duration consistency
out = {
    'status': r.status,
    'flags': r.flags,
}

try:
    import matplotlib.pyplot as plt
    from tess_vetter.api import plot_duration_consistency
    PLOTTING_AVAILABLE = True
except Exception as e:
    PLOTTING_AVAILABLE = False
    out['plotting_error'] = str(e)

if PLOTTING_AVAILABLE and (r.status == 'ok') and getattr(r, 'raw', None) and (r.raw or {}).get('plot_data'):
    run_out_dir, docs_out_dir = sh.artifact_dirs(step_id='06_v03_duration')
    run_path = run_out_dir / 'V03_duration_consistency.png'
    docs_path = (docs_out_dir / 'V03_duration_consistency.png') if docs_out_dir is not None else None

    fig, ax = plt.subplots(figsize=(7, 5))
    plot_duration_consistency(r, ax=ax)
    ax.set_title('V03: Duration consistency')
    fig.tight_layout()

    fig.savefig(run_path, dpi=150, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=150, bbox_inches='tight')
    plt.show()

    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


**Pre-rendered plot (no execution required):** `../artifacts/tutorial_toi-5807-incremental/06_v03_duration/V03_duration_consistency.png`

![V03: Duration consistency](../artifacts/tutorial_toi-5807-incremental/06_v03_duration/V03_duration_consistency.png)


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/06_v03_duration/V03_duration_consistency.png",
  "flags": [],
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/06_v03_duration/V03_duration_consistency.png",
  "status": "ok"
}
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Flags:** none.
- **Result:** observed duration **4.046 h** vs expected **6.397 h** (ratio ≈ **0.632**).
- **Why it’s useful:** flags potential geometry/assumption issues early (host mismatch, grazing geometry, or eccentricity).
- **Interpretation:** shorter-than-expected duration alone is not disqualifying; treat as context for later localization/robustness checks.
- **Next step:** V04 (depth stability / epoch-to-epoch consistency).

</details>
